In [1]:
import numpy as np
import cv2
from scipy.misc import imresize
#from moviepy.editor import VideoFileClip
from IPython.display import HTML
from keras.models import load_model

Using TensorFlow backend.


In [3]:
# Load Keras model
#model = load_model('model/lanes/trained_lane_model_3_layer_new.h5')
model = load_model('model/lanes/trained_lane_model_3_layer_new_2.h5')


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [15]:
def draw_lanes(image, label=np.array([]), predshape=(80, 160, 3), shape=(1080, 1632, 3)):
        
    # Resize images
    small_img = imresize(image, predshape)    
    small_img = np.array(small_img)
    small_img = small_img[None,:,:,:]
            
    # Make prediction, if there is no given label
    if(label.shape != (0,)):
        lane_drawn = label
        
    else:

        prediction = model.predict(small_img)[0] * 255

        # Generate fake R & G color dimensions, stack with B
        blanks = np.zeros_like(prediction).astype(np.uint8)
        lane_drawn = np.dstack((blanks, blanks, prediction))
            
    # Re-size to match the original image
    lane_image = imresize(lane_drawn, shape)
        
    # Merge the lane drawing onto the original image
    result = cv2.addWeighted(image, 1, lane_image, 1, 0)
        
    return result

In [23]:
import PIL

USELABEL = False

# test how the prediction responds to a given scene
img = PIL.Image.open("images/test/out_scene_965.jpg")

#img = PIL.Image.open("images/two-lane-road.jpg")

img = np.asarray(img)

if USELABEL:

    label = imresize(PIL.Image.open("images/labelled/out_215.jpg"), (80, 160, 3))

    label = np.asarray(label)

    res = draw_lanes(img, label=label, shape=(100, 255, 3))
    
    res = PIL.Image.fromarray(res)

else:

    res = draw_lanes(img, shape=(452, 602, 3))
    # res = draw_lanes(img, shape=(229, 400, 3))
    # res = draw_lanes(img, shape=(293, 440, 3))

    res = PIL.Image.fromarray(res)

res



C:\Users\t-dhgup\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  after removing the cwd from sys.path.
C:\Users\t-dhgup\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


(100, 255, 3)


error: OpenCV(3.4.1) C:\bld\opencv_1520732670222\work\opencv-3.4.1\modules\core\src\arithm.cpp:659: error: (-209) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function cv::arithm_op


In [45]:
import PIL

# test how the prediction responds to a given scene

for i in range(1377):
    
    img = PIL.Image.open("images/scene/out_scene_%d.jpg" % i)

    img = np.asarray(img)

    res = draw_lanes(img, shape=(100, 255, 3))

    res = PIL.Image.fromarray(res)
    
    res.save("images/test_scene/out_scene_%d.jpg" % i)



FileNotFoundError: [Errno 2] No such file or directory: 'images/scene/out_scene_797.jpg'

In [44]:
# output video
output_path = 'videos/test/output_scene.mp4'

# input video
vid = VideoFileClip("images/scene/movie.mp4")

vid_labelled = vid.fl_image(road_lines)
vid_labelled.write_videofile(output_path, audio=False)

[MoviePy] >>>> Building video videos/test/output_scene.mp4
[MoviePy] Writing video videos/test/output_scene.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 797/798 [00:21<00:00, 37.71it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: videos/test/output_scene.mp4 

